# tracktop lib Usage

In [75]:
import sys
import numpy as np
import pandas as pd

sys.path.append('../utils')
from tracktop import *

pivot = 6

## Open dataframes from csv

In [65]:
# Open the input file as a DF
tracks      = pd.read_csv('/home/jfialho/git/track-ml/output/tracks_real_event000001937.csv',
                          header = None, sep = ' ')

# removing 6 fisrt columns information and de last column (real or fake)
tracks_real = tracks.iloc[:,6:]

tracks      = pd.read_csv('/data/track-ml/output/trtomei_fake.csv',header = None)

# removing 6 fisrt columns information and de last column (real or fake)
tracks_fake = tracks.iloc[:,7:-1]

# Replacing 0.0 -> infinite representation
# to move the hits with zeroes to the bottom of the dataframe
tracks_real_sort = tracks_real.replace(0.0, np.PINF).copy()
tracks_fake_sort = tracks_fake.replace(0.0, np.PINF).copy()

print('shape of tracks: ' + str(tracks.shape))
print('shape of tracks_real: ' + str(tracks_real.shape))
print('shape of tracks_fake: ' + str(tracks_fake.shape))

shape of tracks: (10000, 122)
shape of tracks_real: (11872, 114)
shape of tracks_fake: (10000, 114)


In [66]:
def rho_from_xy(x, y):
    return np.sqrt(x * x + y * y)


def eta_from_theta(theta):
    return -np.log(np.tan(theta / 2.0))


def eta_from_xyz(x, y, z):
    theta = np.arctan2(np.sqrt(x * x + y * y), z)
    return eta_from_theta(theta)


def phi_from_xy(x, y):
    return np.arctan2(y, x)


def convert_xyz_to_rhoetaphi(x, y, z):
    return rho_from_xy(x, y), eta_from_xyz(x, y, z), phi_from_xy(x, y)


def convert_rhoetaphi_to_xyz(rho, eta, phi):
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    z = rho * np.sinh(eta)
    return x, y, z


In [87]:
x = 1
y = 2
z = 3

rho, eta, phi = convert_xyz_to_rhoetaphi(x, y, z)

convert_rhoetaphi_to_xyz(rho, eta, phi)


(1.0000000000000002, 2.0, 3.000000000000001)

## Sorting the dataframe using xyz_bsort function

### REAL tracks

In [68]:
for i in range(int(tracks_real_sort.shape[0])):
#for i in range(100):
    xyz_bsort(tracks_real_sort.iloc[i,:])

# Replacing infinite representation -> 0.0    
tracks_real_sort = tracks_real_sort.replace(np.PINF,0.0).copy()



In [107]:
#tracks_real_sort.to_csv('../output/tracks_real_sort_19142129052019.csv', 
#                        encoding='utf-8', index = False, header = False)

In [77]:

dft_size_col = tracks_real_sort.shape[1]
len_xyz = int(dft_siz_col/pivot)

# Initializing lists of indexes
selected_columns_x = np.zeros(len_xyz)
selected_columns_y = np.zeros(len_xyz)
selected_columns_z = np.zeros(len_xyz)

# Generating indexes
for i in range(len_xyz):
    selected_columns_x[i] = int(i*pivot)
    selected_columns_y[i] = int(i*pivot+1)
    selected_columns_z[i] = int(i*pivot+2)
    


In [101]:
#for i in range(tracks_real_sort.shape[0]):
def convert_line_rhoetaphi(df_in, df_out):

    len_xyz = df_in.shape[0] // pivot

    for i in range(len_xyz):
        pivot_tmp = i * pivot
        x = df_in.iloc[pivot_tmp] 
        y = df_in.iloc[pivot_tmp + 1] 
        z = df_in.iloc[pivot_tmp + 2]
        if (x == 0 and y == 0 and z == 0):
            rho, eta, phi = 0, 0, 0
        else:
            rho, eta, phi = convert_xyz_to_rhoetaphi(x, y, z)
            df_out.iloc[pivot_tmp] = rho 
            df_out.iloc[pivot_tmp + 1] = eta
            df_out.iloc[pivot_tmp + 2] = phi


In [102]:
tracks_real_rhoetaphi = tracks_real_sort.copy()

for i in range(10):
    convert_line_rhoetaphi(tracks_real_sort.iloc[i,:], tracks_real_rhoetaphi.iloc[i,:])

In [103]:
tracks_real_rhoetaphi

,6,7,8,9,10,11,12,13,14,15,...,110,111,112,113,114,115,116,117,118,119
0,32.048826,2.750395,1.429958,203.063830,724.063830,0.052112,74.966438,2.773603,1.389086,322.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,32.345557,-0.521061,-2.179450,154.400000,948.400000,0.062087,71.795395,-0.158939,-2.201880,248.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,116.803579,-0.036398,-2.144896,95.636364,1186.363636,0.047403,258.664712,-1.381873,0.868096,313.428571,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,116.803579,-0.036398,-2.144896,95.500000,1186.000000,0.138146,256.076454,-0.193076,0.303863,13.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,32.276525,-2.447549,2.154130,163.461538,459.038462,0.055083,72.499190,-2.330063,2.197599,78.629630,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,32.488324,-1.062462,-0.241271,137.888889,526.111111,0.050875,71.441066,-0.957181,-0.259478,217.571429,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,32.043853,-1.072531,-0.366416,310.000000,526.500000,0.065737,33.526554,-1.064151,-0.365764,53.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [104]:
tracks_real_sort

,6,7,8,9,10,11,12,13,14,15,...,110,111,112,113,114,115,116,117,118,119
0,4.498790,31.731501,249.738998,203.063830,724.063830,0.052112,13.547300,73.732201,598.000000,322.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-18.493999,-26.536900,-17.627100,154.400000,948.400000,0.062087,-42.360699,-57.966801,-11.459200,248.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-63.433601,-98.077797,-4.252390,95.636364,1186.363636,0.047403,167.169998,197.386993,-482.571014,313.428571,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,-63.433601,-98.077797,-4.252390,95.500000,1186.000000,0.138146,244.345001,76.620300,-49.750000,13.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,-17.778200,26.938999,-185.162003,163.461538,459.038462,0.055083,-42.525002,58.717602,-369.067993,78.629630,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,31.547300,-7.762670,-41.388199,137.888889,526.111111,0.050875,69.049500,-18.330099,-79.312798,217.571429,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,29.916700,-11.480400,-41.346600,310.000000,526.500000,0.065737,31.308800,-11.991200,-42.802601,53.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### FAKE tracks

In [105]:
#for i in range(int(tracks_fake_sort.shape[0])):
for i in range(10):
    xyz_bsort(tracks_fake_sort.iloc[i,:])

    # Replacing infinite representation -> 0.0   
tracks_fake_sort = tracks_fake_sort.replace(np.PINF,0.0).copy()

### Ploting dataframe of tracks

**Default data:** A dataframe with one track perline and just 6 hit information 

In [106]:
track_plot(tracks_real_rhoetaphi, n_tracks = 10)

### Ploting using parameters

In [ ]:
track_plot(tracks_fake_sort, track_color = 'blue', n_tracks = 10, title = 'Fake tracks sorted')